In [ ]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime

## Group Assignment
### Team Number: 07
### Team Member Names: Wendi Xue, 
### Team Strategy Chosen: Risk-Free(Market Beat, Market Meet, Risk-Free)

### Loading and filtering valid tickers
In the secret list of tickers that will be used to generate our portfolio, we will take into consideration the following factors:
- If there are any duplicate tickers in the list, we will drop them.
- If there are any tickers in the list that do not exist, using the function info.get() will prevent any KeyErrors
- We only want valid US and Canadian companies that have listed stocks in the markets, so we will only take stocks with United States or Canada as their country and CAD and USD as their currency

In [ ]:
# Load the tickers
df = pd.read_csv('Tickers_Example.csv', header=None) # test with Tickers_Example.csv?
# If there are any duplicate tickers, we will drop them
df = df.drop_duplicates()
ticker_lst = list(df.iloc[:,0])

# Filter out tickers that don't exist; include only valid tickers
valid_tickers_lst = []
for ticker in ticker_lst:
    try: 
        ticker_data = yf.Ticker(ticker) # call up data
        country = ticker_data.info.get('country') 
        currency = ticker_data.info.get('currency')
        if country in {'Canada','United States'} and currency in {'CAD','USD'}: # filters us and canadian tickers and listed stocks
            valid_tickers_lst.append(ticker)
    except Exception as err:
        print(f"Error: {err}")

print(valid_tickers_lst)

## Average daily volume constraint
- If the stock does not have any information on their volume on a specific day, we will drop that day.
- Between October 1, 2024 and September 30, 2025, we will group together the data of the volume by months to see if there are months with less than 18 trading days. If there is we will drop that month when calculating the average daily volume.
- If any stock has an average daily volume below 5,000 shares we will exclude that stock from the list of tickers.

In [ ]:
# Apply a filter to remove all stocks with volumes below 5000 (in the given period)
# period in which we look at the volumes 
volume_start_date = '2024-10-01'
volume_end_date = '2025-09-30' 
filtered_lst = []

for ticker in valid_tickers_lst: # goes through every ticker to filter
    data = yf.download(
        tickers=ticker,
        start=volume_start_date,
        end=volume_end_date
    )
    volume_data = data[['Volume']].dropna() # volume data

    keep_months = pd.DataFrame() # df of all the months with more than 18 trading days
    volume_data['Month'] = volume_data.index.to_period('M') # create new column of index only by (YYYY-MM)
    grouped_month_index = volume_data.groupby(['Month']) # group data by month

    for month, group in grouped_month_index:
        if len(group) >= 18: # if the month has more than 18 trading days
            keep_months = pd.concat([keep_months, group]) # add data of months with more than 18 trading days

    average_daily_volume = keep_months['Volume'][ticker].mean() # calculate average daily volume
    if average_daily_volume >= 5000: # determine if above or below 5000 shares
        filtered_lst.append(ticker) # add to filtered list if greater or equal to 5000 shares

print(filtered_lst)

# Get the daily data over chosen timeframe (2025-10-24 to 2025-10-31 for testing?)
start_date = '2025-10-24' # change to Nov 21 2025
end_date = '2025-10-31' # change to Nov 28 2025

daily_data = yf.download(
    tickers=filtered_lst,
    start=start_date,
    end=end_date)

In [ ]:
# Get/calculate volatility (std), beta, market cap, and sectors

# We'll store all the metrics in this dataframe
metrics_df = pd.DataFrame(columns=['Ticker', 'Volatility', 'Beta', 'MarketCap', 'Sector'])

for ticker in filtered_lst:
    # 1. Get price data for this ticker from the daily_data frame
    # daily_data has a MultiIndex on the columns: (field, ticker)
    try:
        prices = daily_data['Adj Close'][ticker]
    except KeyError:
        # Fallback if Adj Close isn't available
        prices = daily_data['Close'][ticker]

    # 2. Compute daily returns and volatility (std of returns)
    returns = prices.pct_change().dropna()
    volatility = returns.std()

    # 3. Pull beta, market cap, and sector from Yahoo Finance info
    t = yf.Ticker(ticker)
    info = t.info

    beta = info.get('beta', np.nan)
    market_cap = info.get('marketCap', np.nan)
    sector = info.get('sector', 'Unknown')

    # 4. Append to our metrics dataframe
    metrics_df.loc[len(metrics_df)] = [ticker, volatility, beta, market_cap, sector]

# Quick look at the result
print(metrics_df.head())

# Use the weighted scoring algorithm in the doc to provide a score /100 per stock

# After scoring, put all stocks in lists based on sector

## Forming the top 25 stocks dataframe
- To maximize our diversification, we will select the top 25 stocks according to the scores allocated by our scoring algorithm, which is the maximum amount we are allowed to have. This will protect our portfolio from single-company risk.
- Using the concept of diversification, we will also spread our selection of stocks accross all sectors so that our portfolio is not entirely dependent on the how well a specific sector performs. Thus, we will pick the top 5 stocks, or the maximum amount of stocks if there are less than 5, with the best score from each sector and from that, we will pick the top 25 stocks with the best scores. This way we will not have more than 5 stocks per sector.


In [ ]:
# Take the top 5 from each sector (based on their score /100) and put them in a new dataframe
top5_count = 0
sector_top5 = pd.DataFrame() # top 5 tickers of a sector with the best scoring
def sector_top5(df,sector):
    for ticker in df['Ticker'].tolist():
        if df.loc[ticker,'Sector'] == sector:
            top5_count += 1
            if top5_count > 5:
                top5_count = 5
                smallest_score = sector_top5['Score'].min()
                smallest_score_index = df.index(smallest_score)
                if df.loc[ticker,'Score'] > smallest_score:
                    sector_top5.loc[smallest_score_index] = df[ticker]
            else:
                sector_top5 = pd.concat([sector_top5,df[ticker]])

sector_lst = df['Sector'].unique # creates a list of all the sectors
all_sectors_top5 = pd.DataFrame()
for sector in sector_lst:
    df_sector_top5 = sector_top5(df,sector)
    all_sectors_top5 = pd.concat([all_sectors_top5,df_sector_top5])

# Then return the top 25
# sort stocks from highest score to lowest
all_sectors_top5 = all_sectors_top5.sort_values('Score', ascending=False)
top_25_stocks = all_sectors_top5.head(25) # top 25 stocks

## Market Cap Mix: Check for a small-cap
To meet the requirement of at least one small-cap:
- Check if the 25th stock in our dataframe is a small-cap (we will need the last row of the dataframe to be a small-cap for the mininum variance portfolio optimization). If it is we will proceed with the minimum varaicnace portfolio optimization.
- If the last row is not a small-cap, check if there already is a small-cap in the entire top 25 stocks. If there is a small-cap, we will move the first small-cap we find in the dataframe to the last row to fufill the requirement for our minimum variance optimization.
- If our top 25 stocks dataframe does not contain a small-cap, we will take the best scoring small-cap from the entire list of tickers and we will replace the last row of our top 25 stocks with it.

In [ ]:
# check for small cap in the last row of the top 25 stocks as code determining weight of portfolio will consider this
small_cap = 2e9
last_row_market_cap = top_25_stocks['MarketCap'].iloc[-1]
market_cap = top_25_stocks['MarketCap']
if last_row_market_cap < small_cap:
    print("Small cap in the 25th row found")
# move small cap to the last row if there is a small cap in the top 25 stocks
elif (market_cap < small_cap).any():
    for row in top_25_stocks.itertuples():
        if row.MarketCap < small_cap:
            top_25_stocks = top_25_stocks.drop(row)
            top_25_stocks = pd.concat([top_25_stocks,row])
# get the best scoring small cap from the dataframe of the top 5 of all sectors and replace it with the stock of the last row of the top 25 stocks            
else:
    small_cap_df = pd.DataFrame()
    for row in all_sectors_top5.itertuples(): # change to metrics dataframe
        if row.MarketCap < small_cap:
            small_cap_df = pd.concat(small_cap_df,row)
        break
    small_cap_df = small_cap_df.sort_values('Score', ascending=False)
    top_score_small_cap = small_cap_df.iloc[0]
    # replace the last stock of the top 25 with the small cap with the best score
    top_25_stocks.iloc[-1] = top_score_small_cap

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.